In [ ]:
import os

os.getcwd()

In [ ]:
! pip install --upgrade --quiet  langchain langchain_community langchain_milvus

In [3]:
# Load docs
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.smartdatapay.com/conventions/KALICONT000005635624.html")
# loader = WebBaseLoader("https://travail-emploi.gouv.fr/droit-du-travail/les-absences-pour-maladie-et-conges-pour-evenements-familiaux/article/les-conges-pour-evenements-familiaux-et-le-conge-de-deuil")
data = loader.load()

type(data) , data

KeyboardInterrupt: 

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
docs = text_splitter.split_documents(data)

docs[:5]

In [ ]:
# ! pip install -qU ollama

In [16]:
# please pull mxbai-embed-large model localy

# import ollama

# ollama.pull('mxbai-embed-large')
# ollama.pull('llama3')


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


KeyboardInterrupt: 

In [7]:
from langchain_community.embeddings import OllamaEmbeddings

EMBEDDING_MODEL="nomic-embed-text"
# EMBEDDING_MODEL="mxbai-embed-large"

embeddings = (
    OllamaEmbeddings(model=EMBEDDING_MODEL, base_url = 'http://localhost:11434', )
) # by default, uses llama2. Run `ollama pull llama2` to pull down the model

In [ ]:
query_result = embeddings.embed_query(docs[0])

query_result[:5]


In [18]:
from langchain_milvus.vectorstores import Milvus

# The easiest way is to use Milvus Lite where everything is stored in a local file.
# If you have a Milvus server you can use the server URI such as "http://localhost:19530".
# URI = "./milvus_demo.db"
URI = "http://localhost:19530"

vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={"uri": URI},
    drop_old=True,  # Drop the old Milvus collection if it exists
)

In [ ]:
query = "Les congés pour événements familiaux et le congé de deuil"
docs = vector_db.similarity_search(query)

#  Liens utiles

- https://python.langchain.com/v0.2/docs/integrations/vectorstores/milvus/
- https://smith.langchain.com/hub/rlm/rag-prompt-llama?organizationId=9554bb1e-9f64-5618-b733-2e911cbbb9f2&tab=0